# Building Your First RAG ChatBot with LangChain
## Part 5a: From Documents to Intelligent Q&A in 30 Minutes

**Welcome to the finale of our LangChain RAG series!** 🎉

In the previous articles, we learned:
- **Part 1**: [Document Loading](link) - Getting data into your system
- **Part 2**: [Document Splitting](blog_document_splitting.md) - Optimal chunking strategies
- **Part 3**: [Vector Embeddings](blog_vector_embeddings.md) - Semantic understanding
- **Part 4**: [Vector Databases](link) - Storage and retrieval at scale

**Now we bring it ALL together!** 🚀

In this notebook, you'll build a working RAG chatbot that:
- ✅ Answers questions about YOUR documents
- ✅ Shows sources for answers
- ✅ Runs completely locally (free!)
- ✅ Takes ~30 minutes to set up

---

## What is RAG Again?

**RAG (Retrieval Augmented Generation)** = Smart Q&A over your documents

**The Problem:**
- LLMs don't know about YOUR specific documents
- They can't access real-time or proprietary information
- They "hallucinate" when they don't know answers

**The Solution (RAG):**
1. User asks a question
2. System retrieves relevant document chunks
3. LLM generates answer using those chunks
4. Result: Accurate, grounded responses with sources!

---

**💡 By the end:** You'll have a chatbot answering questions about any PDF, website, or text you provide!

## Architecture Overview

```
┌─────────────┐
│  Documents  │ (PDFs, websites, text)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│   Loader    │ (Read documents)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│  Splitter   │ (Chunk into pieces)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│ Embeddings  │ (Convert to vectors)
└──────┬──────┘
       │
       ▼
┌─────────────┐
│ Vector DB   │ (Store in Chroma)
└──────┬──────┘
       │
┌──────┴──────┐
│             │
▼             ▼
User ──► Retriever ──► LLM ──► Answer
Question     (Find       (Generate
             relevant)   response)
```

**Simple, right?** Let's build it step by step!

## Setup: Install Required Packages

In [ ]:
# Install all required packages
pip install -q langchain langchain-community langchain-chroma
pip install -q langchain-huggingface sentence-transformers
pip install -q langchain-openai  # Optional: for OpenAI models
pip install -q pypdf  # For PDF loading
pip install -q chromadb  # Vector database

print("✓ All packages installed successfully!")

In [ ]:
# Diagnostic: Check Python, Torch, GPU, and DLL issues
import sys, os, subprocess, traceback
print('Python executable:', sys.executable)
print('Python version:', sys.version.replace('\n',' '))
print('64-bit Python?:', sys.maxsize > 2**32)
print('\nFirst entries of sys.path:')
for p in sys.path[:10]:
    print('  ', p)

print('\n--- pip show torch (for this Python binary) ---')
subprocess.run([sys.executable, '-m', 'pip', 'show', 'torch'])

print('\n--- pip list (filtered for torch packages) ---')
subprocess.run([sys.executable, '-m', 'pip', 'list', '--format=columns'])

print('\n--- conda list torch (if conda available) ---')
try:
    subprocess.run(['conda', 'list', 'torch'])
except Exception as e:
    print('conda not available or error:', e)

print('\n--- nvidia-smi output (if NVIDIA GPU/drivers installed) ---')
try:
    subprocess.run(['nvidia-smi'])
except Exception as e:
    print('nvidia-smi not found or error:', e)

print('\n--- Attempt to import torch and inspect error (if any) ---')
try:
    import torch
    print('torch import succeeded')
    print('torch version:', getattr(torch, '__version__', 'unknown'))
    print('torch cuda version:', getattr(torch.version, 'cuda', None))
    print('cuda available:', torch.cuda.is_available())
    print('torch lib location:', getattr(torch, '__file__', None))
except Exception as e:
    print('Import torch failed (traceback below):')
    traceback.print_exc()
    print()
    print('Common fixes:')
    print('- Install matching PyTorch build for your CUDA driver or install CPU-only PyTorch')
    print('- Update GPU drivers / CUDA toolkit (if using GPU)')
    print('- Install the Microsoft Visual C++ 2015-2022 Redistributable (x64)')
    print('- Try creating a fresh conda env and install CPU-only PyTorch to isolate the issue')

## Optional: API Keys Setup

**Two options for LLM:**
1. **OpenAI** (paid, best quality) - Requires API key
2. **Local LLM** (free, good enough) - No API key needed

We'll show both approaches!

In [3]:
from langchain.agents import create_agent
import os

# Option 1: Use OpenAI (uncomment if you have API key)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Option 2: We'll use HuggingFace (free, local)
# No API key needed!

print("✓ Configuration ready")


e:\Langchain Training\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Configuration ready


---

# Step 1: Load Documents 📄

First, we need documents to answer questions about. We'll use multiple sources.

## Option A: Load from Sample Text (Quick Start)

In [4]:
from langchain_core.documents import Document

# Sample documents about LangChain and RAG
sample_docs = [
    Document(
        page_content="""LangChain is a framework for developing applications powered by language models. 
        It enables applications that are context-aware and can reason about information. LangChain provides 
        tools for document loading, text splitting, embeddings, vector stores, and chains for building 
        complex LLM applications.""",
        metadata={"source": "langchain_intro.txt", "topic": "LangChain Overview"}
    ),
    Document(
        page_content="""Retrieval Augmented Generation (RAG) is a technique that combines information retrieval 
        with text generation. It works by first retrieving relevant documents from a knowledge base, then 
        passing those documents to a language model to generate accurate, grounded responses. This prevents 
        hallucinations and allows LLMs to answer questions about specific, up-to-date information.""",
        metadata={"source": "rag_explanation.txt", "topic": "RAG"}
    ),
    Document(
        page_content="""Vector embeddings are numerical representations of text that capture semantic meaning. 
        Similar texts produce similar embedding vectors. Common embedding models include OpenAI's text-embedding-3-small, 
        sentence-transformers from HuggingFace like all-MiniLM-L6-v2, and Google's text-embedding-004. 
        Embeddings typically have 384 to 1536 dimensions.""",
        metadata={"source": "embeddings_guide.txt", "topic": "Embeddings"}
    ),
    Document(
        page_content="""Chroma is an open-source embedding database designed for AI applications. 
        It provides fast similarity search, easy integration with LangChain, and persistent storage. 
        Chroma supports metadata filtering and is perfect for building RAG applications. It can run 
        locally or in client-server mode.""",
        metadata={"source": "chroma_db.txt", "topic": "Vector Databases"}
    ),
    Document(
        page_content="""Document splitting is crucial for RAG systems. The RecursiveCharacterTextSplitter 
        is recommended as it intelligently splits on paragraphs, sentences, and words in order. 
        Optimal chunk size is typically 500-1000 characters with 10-20% overlap. The overlap ensures 
        context isn't lost at chunk boundaries.""",
        metadata={"source": "splitting_guide.txt", "topic": "Document Splitting"}
    ),
]

print(f"✓ Loaded {len(sample_docs)} sample documents")
print(f"\nSample content: {sample_docs[0].page_content[:100]}...")

✓ Loaded 5 sample documents

Sample content: LangChain is a framework for developing applications powered by language models. 
        It enables...


## Option B: Load from PDF (Your Own Documents)

In [2]:
# Uncomment to load your own PDF
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/AI_Agents.pdf")
sample_docs = loader.load()
print(f"✓ Loaded {len(sample_docs)} pages from PDF")

#print("💡 To use your own PDF: uncomment above and provide PDF path")

e:\Langchain Training\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Loaded 117 pages from PDF


## Option C: Load from Website

In [ ]:
# Uncomment to load from a website
# from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
# sample_docs = loader.load()
# print(f"✓ Loaded content from website")

print("💡 To load from web: uncomment above and provide URL")

---

# Step 2: Split Documents into Chunks 🔪

We split documents into smaller chunks for better retrieval precision.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Size of each chunk
    chunk_overlap=50,      # Overlap between chunks (preserves context)
    length_function=len,   # How to measure chunk size
    separators=["\n\n", "\n", " ", ""]  # Split on paragraphs, then sentences, then words
)

# Split the documents
splits = text_splitter.split_documents(sample_docs)

print(f"✓ Split {len(sample_docs)} documents into {len(splits)} chunks")
print(f"\nSample chunk:")
print(f"Content: {splits[0].page_content}")
print(f"Metadata: {splits[0].metadata}")
print(f"Chunk size: {len(splits[0].page_content)} characters")

✓ Split 5 documents into 5 chunks

Sample chunk:
Content: LangChain is a framework for developing applications powered by language models. 
        It enables applications that are context-aware and can reason about information. LangChain provides 
        tools for document loading, text splitting, embeddings, vector stores, and chains for building 
        complex LLM applications.
Metadata: {'source': 'langchain_intro.txt', 'topic': 'LangChain Overview'}
Chunk size: 328 characters


**💡 Why splitting matters:**
- Too large: Retrieves irrelevant info, wastes LLM context
- Too small: Loses context, poor answers
- Just right: Precise retrieval, better answers!

---

# Step 3: Create Embeddings 🧮

Convert text chunks into vector embeddings for semantic search.

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize embeddings model (free, runs locally)
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # Fast, 384 dimensions
    #model_name="all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU
)

# Test the embeddings
test_text = "What is LangChain?"
test_embedding = embeddings.embed_query(test_text)

print(f"✓ Embeddings model loaded")
print(f"Model: all-MiniLM-L6-v2")

print(f"Embedding dimensions: {len(test_embedding)}")
print(f"Sample values: {test_embedding[:5]}")

✓ Embeddings model loaded
Model: all-MiniLM-L6-v2
Embedding dimensions: 384
Sample values: [-0.04055781289935112, 0.013319483026862144, 0.019409120082855225, 0.0048216828145086765, -0.06119084730744362]


**Alternative: Use OpenAI Embeddings (if you have API key)**

In [6]:
# Uncomment if using OpenAI
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print("✓ Using OpenAI embeddings")

print("💡 OpenAI embeddings are higher quality but cost ~$0.02 per 1M tokens")

✓ Using OpenAI embeddings
💡 OpenAI embeddings are higher quality but cost ~$0.02 per 1M tokens


---

# Step 4: Create Vector Store with Chroma 🗄️

Store embeddings in Chroma for fast similarity search.

In [7]:
from langchain_chroma import Chroma
import shutil

# Clean up any existing database
shutil.rmtree("./chroma_rag_db", ignore_errors=True)

# Create vector store
print("Creating vector store...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_rag_db",  # Persist to disk
    collection_name="rag_demo"
)

print(f"✓ Vector store created with {len(splits)} chunks")
print(f"✓ Persisted to: ./chroma_rag_db")

# Test retrieval
test_query = "What is RAG?"
test_results = vectorstore.similarity_search(test_query, k=2)

print(f"\nTest Query: '{test_query}'")
print(f"Top result: {test_results[0].page_content[:100]}...")

Creating vector store...
✓ Vector store created with 5 chunks
✓ Persisted to: ./chroma_rag_db

Test Query: 'What is RAG?'
Top result: Retrieval Augmented Generation (RAG) is a technique that combines information retrieval 
        wit...


**💡 What just happened:**
1. Each chunk was converted to a 384-dimensional vector
2. Vectors stored in Chroma with metadata
3. Chroma created an index for fast similarity search
4. Data persisted to disk (can reload later!)

---

# Step 5: Create a Retriever 🔍

Retriever finds the most relevant chunks for a query.

In [8]:
# Create retriever from vector store
retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance (diverse results)
    search_kwargs={
        "k": 3,          # Return top 3 chunks
        "fetch_k": 10    # Fetch 10 candidates for MMR selection
    }
)

print("✓ Retriever created with MMR search")
print("  - Returns: 3 most relevant chunks")
print("  - Strategy: MMR (diverse, non-redundant results)")

# Test retrieval
test_query = "What are AI Agents?"
retrieved_docs = retriever.invoke(test_query)
#retrieved_docs = retriever.get_relevant_documents(test_query)

print(f"\nTest Query: '{test_query}'")
print(f"Retrieved {len(retrieved_docs)} chunks:\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"{i}. {doc.page_content[:80]}...")
    print(f"   Source: {doc.metadata.get('source', 'N/A')}\n")

✓ Retriever created with MMR search
  - Returns: 3 most relevant chunks
  - Strategy: MMR (diverse, non-redundant results)

Test Query: 'What are AI Agents?'
Retrieved 3 chunks:

1. Retrieval Augmented Generation (RAG) is a technique that combines information re...
   Source: rag_explanation.txt

2. Chroma is an open-source embedding database designed for AI applications. 
     ...
   Source: chroma_db.txt

3. Vector embeddings are numerical representations of text that capture semantic me...
   Source: embeddings_guide.txt



In [11]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectorstore.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

**💡 Why MMR?**
- Standard search might return 3 very similar chunks
- MMR balances relevance + diversity
- Result: Better coverage, less redundancy

---

# Step 6: Set Up the LLM 🤖

Choose your LLM: OpenAI (paid) or HuggingFace (free).

## Option A: OpenAI (Best Quality)

In [9]:
# Uncomment if you have OpenAI API key
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
   temperature=0  # 0 = focused, factual responses
)
print("✓ Using OpenAI GPT-4o-mini")

print("💡 OpenAI provides best quality but costs ~$0.002 per request")

✓ Using OpenAI GPT-4o-mini
💡 OpenAI provides best quality but costs ~$0.002 per request


## Option B: HuggingFace (Free, Local)

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
# Use free HuggingFace Inference API
llm = HuggingFaceEndpoint(
    #repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    #repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    temperature=0.1,
    max_new_tokens=512,
)

print("✓ Using HuggingFace: Mistral-7B-Instruct")
print("  - Free to use (rate limited)")
print("  - Good quality for most tasks")

# Test the LLM
test_response = llm.invoke("What is 2+2? Answer in one word.")
print(f"\nTest: {test_response}")

✓ Using HuggingFace: Mistral-7B-Instruct
  - Free to use (rate limited)
  - Good quality for most tasks


StopIteration: 

**💡 Note:** HuggingFace free tier has rate limits. For production, use OpenAI or host your own model.

---

# Step 7: Build the RAG Chain 🔗

This is where the magic happens! Connect retriever + LLM.

In [15]:
from langchain_classic.chains import RetrievalQA 
#from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import PromptTemplate

# Create a custom prompt
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer: Let me help you with that."""

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",  # "stuff" = put all retrieved docs into prompt
    retriever=retriever,
    return_source_documents=True,  # Return sources with answer
    chain_type_kwargs={"prompt": PROMPT}
)

print("✓ RAG Chain created successfully!")
print("  - Retriever: Finds relevant chunks")
print("  - LLM: Generates answer from chunks")
print("  - Returns: Answer + source documents")

✓ RAG Chain created successfully!
  - Retriever: Finds relevant chunks
  - LLM: Generates answer from chunks
  - Returns: Answer + source documents


In [10]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vectorstore.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [12]:
query = "What are vector embeddings?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What are vector embeddings?
================================== Ai Message ==================================

Vector embeddings are numerical representations of text that capture the semantic meaning of the content. They transform words, phrases, or entire documents into fixed-size vectors of numbers, allowing for the comparison of their meanings based on their proximity in the vector space. Similar texts produce similar embedding vectors, which makes it easier to perform tasks like similarity search, clustering, and classification.

Common embedding models include OpenAI's text-embedding-3-small, sentence-transformers from HuggingFace like all-MiniLM-L6-v2, and Google's text-embedding-004. These embeddings typically have dimensions ranging from 384 to 1536, depending on the model used. By representing text in this way, vector embeddings enable various applications in natural language processing and machi

**💡 What's happening in the chain:**
1. User asks question
2. Retriever finds 3 most relevant chunks
3. Chunks inserted into prompt as "context"
4. LLM generates answer using that context
5. Sources returned for verification

---

# Step 8: Test Your RAG ChatBot! 🎉

Let's ask some questions!

In [16]:
def ask_question(question):
    """Ask a question and display answer with sources."""
    print(f"\n{'='*70}")
    print(f"Question: {question}")
    print(f"{'='*70}\n")
    
    result = qa_chain.invoke({"query": question})
    
    print(f"Answer:\n{result['result']}\n")
    print(f"{'='*70}")
    print(f"Sources:")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"\n{i}. {doc.metadata.get('source', 'Unknown')}")
        print(f"   {doc.page_content[:150]}...")
    print(f"{'='*70}")

# Test with sample questions
questions = [
    "What is LangChain?",
    "How does RAG work?",
    "What embedding models are commonly used?",
    "Why is document splitting important?"
]

# Ask the first question
ask_question(questions[0])


Question: What is LangChain?

Answer:
LangChain is a framework for developing applications powered by language models. It enables applications that are context-aware and can reason about information. LangChain provides tools for document loading, text splitting, embeddings, vector stores, and chains for building complex LLM applications.

Sources:

1. langchain_intro.txt
   LangChain is a framework for developing applications powered by language models. 
        It enables applications that are context-aware and can reaso...

2. chroma_db.txt
   Chroma is an open-source embedding database designed for AI applications. 
        It provides fast similarity search, easy integration with LangChain...

3. splitting_guide.txt
   Document splitting is crucial for RAG systems. The RecursiveCharacterTextSplitter 
        is recommended as it intelligently splits on paragraphs, se...


In [ ]:
# Try more questions
ask_question(questions[1])

In [ ]:
# Ask your own question!
custom_question = "What is Chroma and why use it?"  # Change this!
ask_question(custom_question)


Question: What is Chroma and why use it?

Answer:
Chroma is an open-source embedding database designed for AI applications. It provides fast similarity search, easy integration with LangChain, and persistent storage. Chroma supports metadata filtering and is particularly well-suited for building Retrieval-Augmented Generation (RAG) applications. It can be run locally or in a client-server mode, making it versatile for different deployment needs. Using Chroma allows developers to efficiently manage and retrieve embeddings, enhancing the performance of AI models that rely on semantic similarity.

Sources:

1. chroma_db.txt
   Chroma is an open-source embedding database designed for AI applications. 
        It provides fast similarity search, easy integration with LangChain...

2. splitting_guide.txt
   Document splitting is crucial for RAG systems. The RecursiveCharacterTextSplitter 
        is recommended as it intelligently splits on paragraphs, se...

3. embeddings_guide.txt
   Vect

In [19]:
# Ask your own question!
custom_question = "Who is Donald Trump?"  # Change this!
ask_question(custom_question)


Question: Who is Donald Trump?

Answer:
I don't know.

Sources:

1. chroma_db.txt
   Chroma is an open-source embedding database designed for AI applications. 
        It provides fast similarity search, easy integration with LangChain...

2. embeddings_guide.txt
   Vector embeddings are numerical representations of text that capture semantic meaning. 
        Similar texts produce similar embedding vectors. Commo...

3. splitting_guide.txt
   Document splitting is crucial for RAG systems. The RecursiveCharacterTextSplitter 
        is recommended as it intelligently splits on paragraphs, se...


**💡 Notice:**
- ✅ Answers are grounded in your documents
- ✅ Sources are provided for verification
- ✅ No hallucination (answers "I don't know" when info not in docs)

**This is the power of RAG!**

---

# Step 9: Interactive Chat Interface 💬

Let's create a simple chat loop for continuous interaction.

In [21]:
def chat_loop():
    """
    Simple chat interface.
    Type 'quit' or 'exit' to end.
    """
    print("\n" + "="*70)
    print("RAG ChatBot Ready! Ask me anything about the documents.")
    print("Type 'quit' or 'exit' to end the conversation.")
    print("="*70 + "\n")
    
    while True:
        # Get user input
        question = input("\nYou: ").strip()
        
        # Check for exit
        if question.lower() in ['quit', 'exit', 'q']:
            print("\nGoodbye! 👋")
            break
        
        if not question:
            continue
        
        # Get answer
        try:
            result = qa_chain.invoke({"query": question})
            print(f"\nBot: {result['result']}")
            print(f"\n📚 Sources: {', '.join([doc.metadata.get('source', 'N/A') for doc in result['source_documents']])}")
        except Exception as e:
            print(f"\nError: {e}")
            print("Please try again.")

# Uncomment to start chat
chat_loop()

print("💡 Uncomment 'chat_loop()' above to start interactive chat!")


RAG ChatBot Ready! Ask me anything about the documents.
Type 'quit' or 'exit' to end the conversation.


Bot: Vector embeddings are numerical representations of text that capture semantic meaning. They are designed such that similar texts produce similar embedding vectors. Common embedding models include OpenAI's text-embedding-3-small, sentence-transformers from HuggingFace like all-MiniLM-L6-v2, and Google's text-embedding-004. Embeddings typically have dimensions ranging from 384 to 1536.

📚 Sources: embeddings_guide.txt, chroma_db.txt, splitting_guide.txt

Bot: I don't know.

📚 Sources: chroma_db.txt, embeddings_guide.txt, splitting_guide.txt

Bot: I don't know.

📚 Sources: chroma_db.txt, splitting_guide.txt, embeddings_guide.txt

Goodbye! 👋
💡 Uncomment 'chat_loop()' above to start interactive chat!


---

# Bonus: Simple Streamlit UI 🎨

Want a web interface? Create `app.py` with this code:

In [ ]:
streamlit_code = '''
import streamlit as st
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain.chains import RetrievalQA

# Page config
st.set_page_config(page_title="RAG ChatBot", page_icon="🤖")
st.title("🤖 RAG ChatBot")
st.caption("Ask questions about your documents!")

# Load components (cached)
@st.cache_resource
def load_rag_chain():
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma(
        persist_directory="./chroma_rag_db",
        embedding_function=embeddings,
        collection_name="rag_demo"
    )
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1)
    
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

qa_chain = load_rag_chain()

# Chat interface
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# User input
if prompt := st.chat_input("Ask a question..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)
    
    # Get response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            result = qa_chain.invoke({"query": prompt})
            response = result["result"]
            sources = [doc.metadata.get("source", "N/A") for doc in result["source_documents"]]
            
            st.write(response)
            st.caption(f"📚 Sources: {', '.join(set(sources))}")
    
    # Add assistant message
    st.session_state.messages.append({"role": "assistant", "content": response})
'''

# Save to file
with open('streamlit_app.py', 'w') as f:
    f.write(streamlit_code)

print("✓ Created streamlit_app.py")
print("\nTo run:")
print("  pip install streamlit")
print("  streamlit run streamlit_app.py")

---

# What You've Built! 🎉

**Congratulations!** You now have a fully functional RAG chatbot that:

✅ **Loads documents** from multiple sources (PDF, web, text)
✅ **Splits intelligently** using RecursiveCharacterTextSplitter
✅ **Creates embeddings** using HuggingFace (free!)
✅ **Stores in Chroma** with persistent storage
✅ **Retrieves with MMR** for diverse, relevant results
✅ **Generates answers** using LLM (Mistral or GPT)
✅ **Shows sources** for verification
✅ **Has a UI** (Streamlit)

**All in ~30 minutes!**

---

## Performance & Cost

**With Free Options (HuggingFace):**
- 💰 Cost: $0
- ⚡ Speed: 2-5 seconds per query
- 📊 Quality: Good for most use cases

**With OpenAI:**
- 💰 Cost: ~$0.002 per query
- ⚡ Speed: <1 second per query
- 📊 Quality: Excellent

---

## Next Steps 🚀

**You can now:**
1. Replace sample docs with YOUR documents
2. Adjust chunk size/overlap for your use case
3. Try different embedding models
4. Experiment with retrieval parameters
5. Customize the prompt for your domain

**In Part 5b (Coming Next):**
- 🔥 Conversational RAG (chat history)
- 🔥 Advanced retrieval (re-ranking, hybrid search)
- 🔥 Evaluation metrics (measure quality)
- 🔥 Production optimization (caching, streaming)
- 🔥 Error handling & edge cases

---

## Common Issues & Solutions

**Problem: "Rate limit exceeded" with HuggingFace**
- Solution: Use OpenAI or host model locally with Ollama

**Problem: "Slow responses"**
- Solution: Use GPU for embeddings, try smaller chunks

**Problem: "Answers not accurate"**
- Solution: Adjust chunk size, try k=5 instead of k=3, improve prompts

**Problem: "Can't find answer in documents"**
- Solution: Check if info is actually in docs, try similarity search directly

---

## Resources

- [LangChain Documentation](https://python.langchain.com/)
- [Chroma Documentation](https://docs.trychroma.com/)
- [HuggingFace Models](https://huggingface.co/models)
- [My Previous Blogs](#) - Parts 1-4 of this series

---

# Thank You! 🙏

**You've completed the journey:**
1. ✅ Document Loading
2. ✅ Document Splitting
3. ✅ Vector Embeddings
4. ✅ Vector Databases
5. ✅ **Building RAG ChatBot** ← You are here!

**What's next?** Try it with YOUR documents and share your results!

**Questions?** Drop them in the comments below!

**Stay tuned for Part 5b** - Production RAG patterns! 🚀

#LangChain #RAG #AI #MachineLearning #Python #ChatBot #VectorDatabase #LLM